# Querying PDF With Astra and LangChain
### A question-answering demo using Astra DB and LangChain, powered by Vector Search

#### Pre-requisites:

You need a **_Serverless Cassandra with Vector Search_** database on [Astra DB](https://astra.datastax.com) to run this demo. As outlined in more detail [here](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html#_prepare_for_using_your_vector_database), you should get a DB Token with role _Database Administrator_ and copy your Database ID: these connection parameters are needed momentarily.

You also need an [OpenAI API Key](https://cassio.org/start_here/#llm-access) for this demo to work.

#### What you will do:

- Setup: import dependencies, provide secrets, create the LangChain vector store;
- Run a Question-Answering loop retrieving the relevant headlines and having an LLM construct the answer.

## Import Dependencies

In [19]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import yaml
from langchain.text_splitter import CharacterTextSplitter
# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio
from PyPDF2 import PdfReader
from typing_extensions import Concatenate

### ASREA cridentials:
You need to provide `ASTRA_DB_APPLICATION_TOKEN`, `ASTRA_DB_ID`, `OPENAI_API_KEY` in a yaml file under `/tmp/langchain-conf.yml` and populate the information you need as below:
  ```python
  OpenAI:
    ASTRA_DB_APPLICATION_TOKEN: <ASTRA_DB_APPLICATION_TOKEN>
    ASTRA_DB_ID: <ASTRA_DB_ID>
    OPENAI_API_KEY: <OPENAI_API_KEY>
    API_Endpoint: <API_Endpoint>
  ```

In [25]:
# Specify the path to your YAML file
OpenAI_conf = '/tmp/langchain-conf.yml'

# Open the YAML file and load its contents
with open(OpenAI_conf, 'r') as file:
    config = yaml.safe_load(file)


In [30]:
pdf_reader = PdfReader('./Attention Is All You Need.pdf')

In [14]:
pdf_reader.pages

In [35]:
raw_text = ''
for i, page in enumerate(pdf_reader.pages):
  if i<11: # analize the pdf until page 10
    content = page.extract_text()
    if content:
      raw_text+= content
len(raw_text)

33800

### Initialize the connection to your database:

In [36]:
cassio.init(token=config['OpenAI']['ASTRA_DB_APPLICATION_TOKEN'], database_id=config['OpenAI']['ASTRA_DB_ID'])


Create your LangChain vector store ... backed by Astra DB!

In [37]:
llm = OpenAI(openai_api_key=config['OpenAI']['OPENAI_API_KEY'])
embedding = OpenAIEmbeddings(openai_api_key=config['OpenAI']['OPENAI_API_KEY'])